In [1]:
import sys
import os

path = os.path.abspath(".")
sys.path.insert(0, os.path.dirname(path))
sys.path

['/gwpool/users/gpizzati/test_processor/my_processor',
 '/gwpool/users/gpizzati/test_processor/my_processor/notebooks',
 '/gwpool/users/gpizzati/mambaforge/envs/test_uproot/lib/python310.zip',
 '/gwpool/users/gpizzati/mambaforge/envs/test_uproot/lib/python3.10',
 '/gwpool/users/gpizzati/mambaforge/envs/test_uproot/lib/python3.10/lib-dynload',
 '',
 '/gwpool/users/gpizzati/mambaforge/envs/test_uproot/lib/python3.10/site-packages']

In [2]:
from framework import read_events, read_chunks

In [3]:
chunks = read_chunks("../condor/job_0/chunks_job.pkl")

In [4]:
events = read_events(
    **{k: v for k, v in chunks[1].items() if k != "dataset" and k != "xs"}
)

start reading
created events


In [8]:
from modules.lepton_sel import createLepton, leptonSel

In [9]:
from data.common import LeptonSel

In [5]:

events = createLepton(events)

In [7]:
events.Lepton.electronIdx

<Array [[None, None, None, None], ..., [0, ...]] type='100000 * var * ?int64'>

In [10]:
ElectronWP = LeptonSel.ElectronWP["Full2018v9"]
MuonWP = LeptonSel.MuonWP["Full2018v9"]

In [11]:
import awkward as ak

In [12]:
ele_mask = abs(events.Lepton.pdgId) == 11
mu_mask = abs(events.Lepton.pdgId) == 13

electron_col = events.Electron[ak.mask(events.Lepton.electronIdx, ele_mask)]
muon_col = events.Muon[ak.mask(events.Lepton.muonIdx, mu_mask)]

for wp in ElectronWP["FakeObjWP"]:
    comb = ak.ones_like(electron_col.pt) == 1.0
    for key, cuts in ElectronWP["FakeObjWP"][wp]["cuts"].items():
        tmp1 = eval(key.replace("[LF_idx]", ""))
        tmp2 = ak.ones_like(electron_col.pt) == 1.0
        for cut in cuts:
            tmp2 = tmp2 & eval(cut.replace("[LF_idx]", ""))
        comb = comb & (~tmp1 | tmp2)
    comb = ak.values_astype(comb, bool)
    events[("Lepton", "ele_isLoose")] = ak.where(ele_mask, comb, False)

for wp in MuonWP["FakeObjWP"]:
    comb = ak.ones_like(muon_col.pt) == 1.0
    for key, cuts in MuonWP["FakeObjWP"][wp]["cuts"].items():
        tmp1 = eval(key.replace("[LF_idx]", ""))
        tmp2 = ak.ones_like(muon_col.pt) == 1.0
        for cut in cuts:
            tmp2 = tmp2 & eval(cut.replace("[LF_idx]", ""))
        comb = comb & (~tmp1 | tmp2)
    comb = ak.values_astype(comb, bool)
    events[("Lepton", "mu_isLoose")] = ak.where(mu_mask, comb, False)


In [18]:
ak.is_none(events.Lepton.mu_isLoose | events.Lepton.ele_isLoose, axis=1)

<Array [[False, False, False, False], ..., [...]] type='100000 * var * bool'>

In [19]:
events = leptonSel(events)

In [20]:
events.Lepton.fields

['pt',
 'eta',
 'phi',
 'mass',
 'pdgId',
 'electronIdx',
 'muonIdx',
 'ele_isLoose',
 'mu_isLoose',
 'isLoose',
 'isTightElectron_mvaFall17V2Iso_WP90',
 'isTightMuon_cut_Tight_HWWW']

In [23]:
ak.any(ak.is_none(events.Lepton.isTightMuon_cut_Tight_HWWW, axis=1))

False

In [27]:
events[('Lepton', 'isTight')] = events.Lepton.isTightMuon_cut_Tight_HWWW | events.Lepton.isTightElectron_mvaFall17V2Iso_WP90

In [29]:
events = events[ak.num(events.Lepton) >= 2]

In [31]:
events.Lepton.isTight[:, 0] & events.Lepton.isTight[:, 1]

<Array [False, False, True, ..., False, False, False] type='67541 * ?bool'>

In [33]:
events.Lepton.isTight

<Array [[True, False, True, False], ..., [...]] type='67541 * var * ?bool'>

In [36]:
ak.any(events.Lepton.isTight != (events.Lepton.isLoose & events.Lepton.isTight))

True

In [49]:

wrongLeptons = events.Lepton[events.Lepton.isTight != (events.Lepton.isLoose & events.Lepton.isTight)]

In [50]:
wrongLeptons = wrongLeptons[ak.num(wrongLeptons)>=1]

In [51]:
wrongLeptons

<Array [[{pt: 6.81, eta: -0.165, ...}], ...] type='3482 * var * ?Momentum4D...'>

In [40]:
events[-4].Lepton.isLoose

<Array [True, False] type='2 * bool'>

In [41]:
events[-4].Lepton.isTight

<Array [True, True] type='2 * ?bool'>

In [48]:
abs(events[-4].Electron.eInvMinusPInv)<0.014

<Array [True, False] type='2 * bool'>

In [42]:
events[-4].Lepton[1]

<Record {pt: 40.5, eta: -2.06, phi: -1.72, ...} type='Momentum4D[pt: float3...'>